In [2]:
import os
from operator import itemgetter
from typing import Dict, List, Optional, Sequence

# import weaviate
from fastapi import FastAPI
from fastapi.middleware.cors import CORSMiddleware
from langchain_anthropic import ChatAnthropic
from langchain_community.chat_models import ChatCohere
from langchain_community.vectorstores import Weaviate
from langchain_core.documents import Document
from langchain_core.language_models import LanguageModelLike
from langchain_core.messages import AIMessage, HumanMessage
from langchain_core.output_parsers import StrOutputParser, JsonOutputParser
from langchain_core.prompts import (
    ChatPromptTemplate,
    MessagesPlaceholder,
    PromptTemplate,
    HumanMessagePromptTemplate,
)
from langchain_core.pydantic_v1 import BaseModel
from langchain_core.retrievers import BaseRetriever
from langchain_core.runnables import (
    ConfigurableField,
    Runnable,
    RunnableBranch,
    RunnableLambda,
    RunnablePassthrough,
    RunnableSequence,
    chain,
)

from langchain_core.output_parsers import JsonOutputParser
from langchain_core.pydantic_v1 import BaseModel, Field

from langchain_fireworks import ChatFireworks
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_openai import ChatOpenAI
from langsmith import Client

import pandas as pd

from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import FAISS

c:\Users\Admin\anaconda3\envs\llm-env\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
def pretty_print_docs(docs):
    print(
        f"\n{'-' * 100}\n".join(
            [
                f"Document {i+1}:\n\n{d.page_content}\nMetadata: {d.metadata}"
                for i, d in enumerate(docs)
            ]
        )
    )

In [4]:
from langchain.retrievers import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import FlashrankRerank
from langchain_openai import ChatOpenAI


from typing import TYPE_CHECKING, Dict, Optional, Sequence

from langchain_core.callbacks.manager import Callbacks
from langchain_core.documents import BaseDocumentCompressor, Document
from langchain_core.pydantic_v1 import Extra, root_validator


In [41]:
df = pd.read_csv("./data_evaluation.csv")
df.sample(1)

,document_index,question,answer
3,9,What items do Rudolf's family take from the Je...,"Rudolf's wife, Hedwig, often receives luxury a..."


In [ ]:
import getpass
import os
os.environ["OPENAI_API_KEY"] = "API_KEY"

from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings()

In [7]:
from langchain_openai import ChatOpenAI

# LLM
llm = ChatOpenAI(model_name="gpt-4o-mini", temperature=0)

In [8]:
# Index
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import FAISS


embed_model = OpenAIEmbeddings()
vectorstore = FAISS.load_local(
    "./RAG_chatbot/",
    OpenAIEmbeddings(),
    allow_dangerous_deserialization=True,
)


retriever = vectorstore.as_retriever(search_kwargs={"k": 20, "threshold": 0.5})
retriever

VectorStoreRetriever(tags=['FAISS', 'OpenAIEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x000001928669F150>, search_kwargs={'k': 20, 'threshold': 0.5})

In [1]:
from langchain.retrievers import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import FlashrankRerank

# Ensure flashrank is installed
try:
    from flashrank import Ranker, RerankRequest
except ImportError:
    raise ImportError(
        "Could not import flashrank python package. "
        "Please install it with `pip install flashrank`."
    )

# Call update_forward_refs to resolve forward references
FlashrankRerank.update_forward_refs()

In [9]:
compressor = FlashrankRerank()
compression_retriever = ContextualCompressionRetriever(
    base_compressor=compressor, base_retriever=retriever
)

In [10]:
# Setup memory for contextual conversation
from langchain.chains import RetrievalQA

chain = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=compression_retriever,
    # memory=memory,
    return_source_documents=True,
    verbose=False,)

from langchain_core.runnables import RunnablePassthrough
from operator import itemgetter

# RAG
template = """Answer the following question based on this context:

{context}

Question: {question}
"""

prompt = ChatPromptTemplate.from_template(template)

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)



In [11]:
result = chain.invoke("What did the goblins say?")

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


In [12]:
result

{'query': 'What did the goblins say?',
 'result': "I don't know.",
 'source_documents': [Document(page_content='Alan Wake 2\nWhy the hell did you kill Casey? What the hell were you thinking, man?\nThis article or a section of this article will contain full, or partial plot spoilers of an Alan Wake game or any other piece of media related to the franchise.\nYou have been warned...\n\n"This story... is a monster. And monsters wear many faces."\n― Alan Wake\n\nAlan Wake 2 (stylized as Alan Wake II) is a survival-horror video game that released on 27 October 2023 on digital storefronts for the PlayStation 5, Xbox Series X|S and PC through the Epic Games Store. It is the sequel to 2010\'s Alan Wake. The game had an expansion DLC released on June 8, 2024 called Night Springs and a second expansion DLC, The Lake House, is slated to be released sometime later. The game has four difficulties: Story, Normal, Hard, and Nightmare, the fourth of which is only available in "The Final Draft" mode. Th

In [16]:
contexts = []
answers = []
for i in range(0, len(df)):
    print(i)
    a = []
    question = df.iloc[i, 1]
    try:
        response = chain.invoke(question)
        result = response['result']
        source_documents = response['source_documents']
        answers.append(result)
        string_check = "" 
        for doc in source_documents:
            # print(doc[0].page_content)
            if doc.page_content not in string_check:
                string_check += doc.page_content
                a.append(doc.page_content)
        contexts.append(a)
    except:
        contexts.append("Error")
        answers.append("Error")
        continue

0


INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


1


INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


2


...

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


18


INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


19


INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


In [17]:
contexts[0], answers[0]

(['---The Paths through the Underground/Underdark---(9 days of travel)\nWandering through the dark tunnels, the rushing sounds of the underground river begin to fade as it diverges from the cavern. You walk on for miles, the smell of hard water and wet earth. Natural chambers and cavern passways are chained together by the stretches of burrowed earth left in the wake of this massive worm-like creature. Clusters of crystal and other beautiful minerals occasionally line the walls and ceilings of the chambers, glittering with the little light you have to shove back the darkness.\n\nDay 1 goes without issue... sleep.\n\nDay 2 – Ropers\nAfter a few miles of winding tunnel, you emerge in a smaller grotto of stalactites and stalagmites dripping with condensation. Unsure if the same underground river, or another water source, is nearby, you can see quite a bit of ground water does funnel down into this area. Seeking the next burrowed entrance left by the Kryn...\n---ENCOUNTER – Ropers x 2---\n

In [42]:
import json
df=df.rename(columns={
    "answer": "ground_truth"
})
df['contexts'] = contexts
df['answer'] = answers
# df['contexts'] = df['contexts'].apply(lambda x: json.loads(x))
df = df.iloc[:,1:]
df

,question,ground_truth,contexts,answer
0,What kind of model is the bling-phi-3 model,The bling-phi-3 model is the newest and most a...,[---The Paths through the Underground/Underdar...,The bling-phi-3 model is a small CPU-based RAG...
1,What are the advantages and disadvantages of t...,The advantage of BM25 is that it is efficient....,[manicfesto proposals\n26m tonnes of waste pla...,I don't know.
2,Who was Duke Stelmane?,Duke Stelmane was a major figure of the Knight...,[The Emperor is a mind flayer who appears in B...,Duke Stelmane was a major figure associated wi...
3,What items do Rudolf's family take from the Je...,"Rudolf's wife, Hedwig, often receives luxury a...",[The best sci-fi and fantasy books of 2023\nIt...,I don't know.
4,What are the rules for developing general purp...,General purpose AI models that were trained us...,[Why do we need to regulate the use of Artific...,The AI Act imposes certain obligations on prov...
5,What can moss be used for?,Harvesting moss gives 1 foraging exp per moss ...,[Why do we need to regulate the use of Artific...,I don't know.
6,In what contexts is BERT mentioned?,It is mentioned that for BERT large during tra...,[Why do we need to regulate the use of Artific...,I don't know.
7,What enemies are encountered in the second enc...,26 kobolds and 1 kobold inventor are encounter...,[Bullet Kin\nBullet Kin are one of the most co...,"In the second encounter, you will face 26 Kobo..."
8,What colour is Nan-E?,No answer,[The best sci-fi and fantasy books of 2023\nIt...,I don't know.
9,How do sets in Python compare to sets in Gleam?,No answer,[How to Maximize Your Impact as a Data Scienti...,I don't know.


In [48]:
df.to_csv('./RAG_Rank_responses.csv', index=False)

In [44]:
df.iloc[16, :]

question                              What do keybullet kin drop?
ground_truth                 Keybullet kin drop a key upon death.
contexts        [Why do we need to regulate the use of Artific...
answer                                              I don't know.
Name: 16, dtype: object

In [45]:
from datasets import Dataset

In [46]:
dataset = Dataset.from_pandas(df)

from ragas.metrics import (
    faithfulness,
    answer_relevancy,
    context_relevancy,
    context_recall,
    context_precision
)

from ragas import evaluate

result = evaluate(
    dataset,
    llm=llm,
    raise_exceptions=False,
    callbacks=None,
    is_async=False,
    embeddings=OpenAIEmbeddings(),
    metrics=[
        faithfulness,
        answer_relevancy,
        context_relevancy,
        context_precision,
        context_recall,
    ],
)

result

Evaluating:   0%|          | 0/100 [00:00<?, ?it/s]INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.ope

{'faithfulness': 0.3838, 'answer_relevancy': 0.3611, 'context_relevancy': 0.0033, 'context_precision': 0.2500, 'context_recall': 0.3312}

In [47]:
result.to_pandas().to_csv('./RAG_Rank_evaluations.csv', encoding='utf-8', index=False)